In [1]:
library(tidyverse)
library(tidymodels)
library(modeltime)
library(timetk)
library(vroom)
library(embed)
library(bonsai)
library(lightgbm)

# Load the training and testing datasets
train_data <- vroom::vroom("/kaggle/input/demand-forecasting-kernels-only/train.csv")
test_data <- vroom::vroom("/kaggle/input/demand-forecasting-kernels-only/test.csv")

# Determine the number of unique stores and items
num_stores <- max(train_data$store)
num_items <- max(train_data$item)

# Define the recipe for preprocessing
sales_recipe <- recipe(sales ~ ., data = train_data) %>%
  step_date(date, features = c("dow", "month", "decimal", "doy", "year")) %>%
  step_range(date_doy, min = 0, max = pi) %>%
  step_mutate(sin_doy = sin(date_doy), cos_doy = cos(date_doy)) %>%
  step_lencode_mixed(all_nominal_predictors(), outcome = vars(sales)) %>%
  step_rm(date, item, store) %>%
  step_normalize(all_numeric_predictors())

# Set up the boosted tree model
boosted_tree_model <- boost_tree(tree_depth = 2, 
                                 trees = 1000, 
                                 learn_rate = 0.01) %>%
  set_engine("lightgbm") %>%
  set_mode("regression")

# Combine the recipe and model into a workflow
boosted_workflow <- workflow() %>%
  add_recipe(sales_recipe) %>%
  add_model(boosted_tree_model)

# Iterate over each store and item to train and forecast
for (store_id in 1:num_stores) {
  for (item_id in 1:num_items) {
    
    # Filter the data for the current store-item combination
    training_subset <- train_data %>%
      filter(store == store_id, item == item_id)
    testing_subset <- test_data %>%
      filter(store == store_id, item == item_id)
    
    # Fit the workflow and make predictions
    trained_model <- boosted_workflow %>%
      fit(data = training_subset)
    predictions <- predict(trained_model, new_data = testing_subset) %>%
      bind_cols(testing_subset) %>%
      rename(sales = .pred) %>%
      select(id, sales)
    
    # Append the predictions to the final output
    if (store_id == 1 && item_id == 1) {
      final_predictions <- predictions
    } else {
      final_predictions <- bind_rows(final_predictions, predictions)
    }
  }
}

# Write the predictions to a CSV file
vroom_write(x = final_predictions, path = "./submission.csv", delim = ",")


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     


── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


── Attaching packages ────────────────────────────────────── tidymodels 1.2.0 ──



✔ broom        1.0.6      ✔ rsample      1.2.1 
✔ dials        1.2.1      ✔ tune         1.2.1 
✔ infer        1.0.7      ✔ workflows    1.1.4 
✔ modeldata    1.4.0      ✔ workflowsets 1.1.0 
✔ parsnip      1.2.1      ✔ yardstick    1.3.1 
✔ recipes      1.0.10     



── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Search for functions across packages at https://www.tidymodels.org/find/



...


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 45000 Columns: 4


── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (3): id, store, item
date (1): date



ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Warning message:
“The `file` argument of `vroom_write()` is deprecated as of vroom 1.5.0.”
